In [1]:
import numpy as np
import networkx as nx
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from qiskit import IBMQ

In [2]:
#IBMQ.enable_account(token=', hub='ibm-q', group='open', project= 'main')

C:\Users\fege9\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [3]:
def gas(n,B,C=None, backend='simulator'):

    provider = IBMQ.get_provider(hub='ibm-q') #getting the appropriate backend from IBM
    if backend=='melbourne':
        backend=provider.get_backend('ibmq_16_melbourne')
    elif backend=='simulator':
        backend=provider.get_backend('ibmq_qasm_simulator')
    
    # create a QUBO
    qubo = QuadraticProgram()
    for i in range(0,n):
        qubo.binary_var('x{}'.format(i))

    qubo.maximize(linear=C, quadratic=B)
    
    grover_optimizer = GroverOptimizer(2*n, num_iterations=10, quantum_instance=backend)
    results = grover_optimizer.solve(qubo)
    
    ex_time=0
    job_id = backend.jobs()[0].job_id() #get time info for latest job
    #alternative: consider last several jobs that represent subproblem
    job = backend.retrieve_job(job_id)
    time_steps=job.time_per_step()
    RUNTIME = time_steps["COMPLETED"] - time_steps["RUNNING"]
    ex_time=RUNTIME.total_seconds()
        
    ex_time=ex_time*1000 #time in milliseconds
    
    
    return results.x, ex_time

In [4]:
B=nx.modularity_matrix(nx.barbell_graph(4,0))
n=8
gas(n,B)

(array([0, 0, 1, 1, 1, 1, 1, 1]), 104377.0)